In [9]:
import numpy as np
import pandas as pd
from utils import get_data, get_preds, get_table, plot_rsme, other_stats, add_intervals_to_test, plot_interval_check, plot_interval_sizes
np.random.seed(2024)

train_yr, test_yr = [2022], [2023]
train_bos, test_bos = get_data(racename="bos", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=True)
train_nyc, test_nyc = get_data(racename="nyc", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=True)
train_chi, test_chi = get_data(racename="chi", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=True)
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [10]:
model_info = [
    # ("no_curr_pace", "stan_results/params_bos1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    # ("with_curr_pace", "stan_results/params_bos2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
    # ("rstan2d", "stan_results/params_bos0d.csv", ["total_pace", "curr_pace", "prop"], ["beta[1]", "beta[2]", "beta[3]"], True),
    ("no_curr_pace", "stan_results/params_nyc1.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    ("with_curr_pace", "stan_results/params_nyc2.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
]
train, test = train_bos, test_bos # train_bos, test_bos

models = {name: pd.read_csv(path) for (name, path, feats, betas, pleft) in model_info}
mpreds = {name: get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name)
           for (name, path, feats, betas, pleft) in model_info}

test2 = get_table(test, mpreds)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,extrap,no_curr_pace,with_curr_pace
0,290180,5K,3.885004,3.885004,3.947516,46,M,2023,0.118497,0.881503,1,0.460363,46,2.866550,18.300383,18.206048
1,292991,5K,3.526093,3.526093,3.701965,27,F,2023,0.118497,0.881503,0,0.417833,0,9.475033,28.212133,28.164889
2,302285,5K,3.240441,3.240441,3.141379,33,F,2023,0.118497,0.881503,0,0.383984,0,-6.843717,15.372122,15.381486
3,288672,5K,4.170142,4.170142,4.106569,31,M,2023,0.118497,0.881503,1,0.494151,31,-2.610650,10.796908,10.677686
4,311517,5K,2.507523,2.507523,2.259680,59,M,2023,0.118497,0.881503,1,0.297135,59,-30.760567,6.778015,7.094876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,309450,40K,2.057613,2.620202,2.550317,70,M,2023,0.947980,0.052020,1,1.950575,70,-7.354638,-6.455931,-4.008110
7996,292271,40K,3.576538,3.767543,3.762036,45,M,2023,0.947980,0.052020,1,3.390485,45,-0.273202,0.543736,0.151481
7997,294978,40K,2.906977,3.564745,3.561361,50,M,2023,0.947980,0.052020,1,2.755755,50,-0.187460,0.649577,2.226922
7998,303895,40K,2.871913,3.051106,3.046350,43,F,2023,0.947980,0.052020,0,2.722515,0,-0.359813,0.521812,0.011451


In [11]:
# s_data =pd.DataFrame([[4, 0.2, .5], [3, 0.5, .75]], columns=["total_pace", "prop", "propleft"])
# get_preds(s_data, models['rstan2c'], feats_lis = ["total_pace", "prop"], beta_lis = ["beta[1]", "beta[2]"], full=True)

In [12]:
labels = ["no_curr_pace", "with_curr_pace", "extrap"]
rmse_table = plot_rsme(test2, labels, "all_errors")
rmse_table["no_curr_pace_"] = 1 - (rmse_table["no_curr_pace"] / rmse_table["extrap"])
rmse_table["with_curr_pace_"] = 1 - (rmse_table["with_curr_pace"] / rmse_table["extrap"])
rmse_table

File saved: analysis/all_errors.png


,no_curr_pace,with_curr_pace,extrap,no_curr_pace_,with_curr_pace_
dist,,,,,
5K,19.792916,19.833731,17.811366,-0.111252,-0.113543
10K,15.349291,15.115139,16.750519,0.083653,0.097632
15K,12.559965,12.244340,15.476271,0.188437,0.208831
20K,10.476785,10.126950,13.455779,0.221391,0.247390
25K,8.270555,7.765108,11.340210,0.270688,0.315259
30K,6.400876,5.921389,8.156740,0.215265,0.274050
35K,4.271132,3.668634,4.644470,0.080383,0.210107
40K,1.240669,0.930461,1.307595,0.051182,0.288418


In [13]:
other_stats(test2[labels], test2["finish"])

,no_curr_pace,with_curr_pace,extrap
0,11.313880,11.090580,12.433882
1,0.928021,0.930834,0.913065


In [14]:
mpreds2 = {name: (42195 / 60) / get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name, full=True)
           for (name, path, feats, betas, pleft) in model_info}
# model_preds2["new1"] = (42195 / 60) / get_preds(test, pd.read_csv("stan_results/params_bos1.csv"), feats_lis=['total_pace', 'curr_pace', 'prop'], beta_lis=['beta.1', 'beta.2', 'beta.3'], propleft=True, full=True)

In [15]:
pred_names = labels[:2]
intervals_tbl = add_intervals_to_test(test2, mpreds2, pred_names)
i_check = plot_interval_check(intervals_tbl, pred_names)
i_sizes = plot_interval_sizes(intervals_tbl, pred_names)

File saved: analysis/interval_check.png
File saved: analysis/interval_sizes.png


<Figure size 4000x4000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>